In [23]:
import pandas as pd
import numpy as np
from utils import *
import os
import fitz
from pdf2image import convert_from_path
import pytesseract
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pdfplumber

In [2]:
os.chdir('/Users/sooyeonkim/Desktop/TenStrands')

In [3]:
master = pd.read_excel('District Level Master Spreadsheet-UCB Policy Analysis - BP_AR 3511.1.xlsx')

In [4]:
master['AR3511.1 PDF Name'] = master['District Name'].str.replace(' ', '_') + '_' + master['County'].str.replace(' ', '_') + '_AR3511.1.pdf'

In [5]:
# load  dataset that contains paths to PDFs
df = master

# initialize columns to store PDF text
df['AR3511.1: PDF Text'] = np.nan

# set the columns to object type explicitly to store strings
df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].astype('object')

In [6]:
# base paths
base_path = "/Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/AR_3511.1/"

# constructing full paths
master['AR3511.1: Path to PDF'] = base_path + master['AR3511.1 PDF Name'].astype(str)

In [7]:
def extract_text_from_pdf(file_path):
    """Extracts text from a PDF file."""
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ''.join([page.extract_text() or '' for page in pdf.pages])
        return text
    except Exception as e:
        print(f"Failed to process {file_path}: {e}")
        return None  

In [8]:
for index, row in df.iterrows():
    if pd.notna(row['AR3511.1: Path to PDF']):
        ar_text = extract_text_from_pdf(row['AR3511.1: Path to PDF'])
        df.at[index, 'AR3511.1: PDF Text'] = ar_text

Failed to process /Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/AR_3511.1/Albany_City_Unified_Alameda_AR3511.1.pdf: [Errno 2] No such file or directory: '/Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/AR_3511.1/Albany_City_Unified_Alameda_AR3511.1.pdf'
Failed to process /Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/AR_3511.1/Fremont_Unified_Alameda_AR3511.1.pdf: [Errno 2] No such file or directory: '/Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/AR_3511.1/Fremont_Unified_Alameda_AR3511.1.pdf'
Failed to process /Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/AR_3511.1/Sunol_Glen_Unified_Alameda_AR3511.1.pdf: [Errno 2] No such file or directory: '/Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/AR_3511.1/Sunol_Glen_Unified_Alameda_AR3511.1.pdf'
Failed to process /Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/AR_3511.1/Biggs_Unified_Butte_AR3511.1.pdf:

In [10]:
def clean_pdf_text(text):
    text = text.lower() # Lowercase text 
    text = re.sub(r'boardpolicymanual', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\w+unifiedschooldistrict\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\d{1,}\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'policy reference disclaimer.*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'supporting documents.*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'state references.*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'education code.*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'legal reference.*', '', text, flags=re.IGNORECASE | re.DOTALL) # regex to remove 'legal reference' and everything after


    tokens = word_tokenize(text) # Tokenize text
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens if re.sub(r'[^\w\s]', '', token)] # regex to match punctuation
    stop_words = set(stopwords.words('english')) # stop words
    tokens = [word for word in tokens if word not in stop_words] # removing stop words 
    lemmatizer = WordNetLemmatizer() 
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # lemmetize 
    cleaned_text = ' '.join(tokens) # join tokens back into a string
    
    return cleaned_text

In [11]:
df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].astype(str)

df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].apply(clean_pdf_text)


In [39]:
df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].str.replace('rreegguullaattiioonn', '', regex=False)
df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].str.replace('iinntteeggrraatteedd', '', regex=False)
df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].str.replace('wwaassttee', '', regex=False)
df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].str.replace('oorriiggiinnaall', '', regex=False)
df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].str.replace('rreevviieewweedd', '', regex=False)
df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].str.replace('aaddoopptteedd', '', regex=False)
df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].str.replace('ddaattee', '', regex=False)
df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].str.replace('llaasstt', '', regex=False)
df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].str.replace('mmaannaaggeemmeenntt', '', regex=False)
df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].str.replace('ssttaattuuss', '', regex=False)
df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].str.replace('oorriiggiinnaall', '', regex=False)
df['AR3511.1: PDF Text'] = df['AR3511.1: PDF Text'].str.replace(r'simblieboard\w+\s*', '', regex=True)

In [41]:
df.to_csv('/Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/cleaned_data/AR_3511.1_cleaned.csv', index='False')